# Import relevant libraries and packages. 

In [1]:
import nidaqmx
from nidaqmx import Task
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import serial
from serial.tools import list_ports
import time
import datetime
import scipy
from scipy.optimize import curve_fit
import warnings
import matplotlib
%matplotlib notebook

ModuleNotFoundError: No module named 'serial'

# Define Functions 

Define the functions that will be used to control the syringe pump and acquire data

In [2]:
def get_device_port():
    
    """
    'get_device_port'  obtains the pump device port which is usually the highest device port

    Returns: 
        Device Port
    """
    
    #Iterate through list in reverse to get highest device COM port
    for comport in reversed(list_ports.comports()):
        device_name = comport.device
        with serial.Serial(device_name, timeout=3) as ser: 
            ser.write(b'/1?\r')
            reading=ser.read_until(b'\r')
            if(reading):
                return device_name
    raise IOError("Syringe not found")


In [3]:
def initialize_and_fill_and_expel_backlash():
    
    ''''
    This function initializes the pump and fills the syringe. 
    It then releases 1000 steps into the output syringe to remove backlash. 
    The pump must be empty before this is called. 
    '''
    
    print('filling syringe')
    
    device_port = get_device_port()
    with serial.Serial(device_port, timeout=5) as ser:
        ser.write(b'/1ZN1R\r')
        time.sleep(5)
        ser.write(b'/1IA12000R\r')
        time.sleep(5)
        ser.write(b'/1OD1000R\r')
        
        # Wait 5 seconds. We need at least 5 seconds before the syringe pump can respond to the next command
        time.sleep(5)
        

In [ ]:
def expel_and_close():
    
    '''
    This function expels the untitrated amount to waste
    '''
    
    print('emptying syringe')
    
    device_port = get_device_port()
    with serial.Serial(device_port, timeout=5) as ser:
        ser.write(b'/1OA0R\r')
    
    # Wait 3 seconds. We need at least 3 seconds before the syringe pump can respond to the next command
    time.sleep(3)
    

In [ ]:
def add_titrant(num_steps):
    
    '''
    This function adds some amount of titrant to the syringe. 
    
    The function takes the argument:
        num_steps = number of steps in the pump motor which detemines amount of titrant added
    '''
    print('adding titrant')
   
    device_port = get_device_port()
    to_write = '/1OD' + str(num_steps) + 'R\r'
    
    with serial.Serial(device_port, timeout=5) as ser:
        ser.write(to_write.encode('utf-8'))
    
    # Wait 3 seconds. We need at least 3 seconds before the syringe pump can respond to the next command
    time.sleep(3)
    

In [ ]:
def get_potential_measurement(samp_rate, samp_num):
    
    '''
    'get_voltage_measurement' is identical to E1. Each acquisition samples the potential some number of times at some sampling rate.
    
    The function takes two arguments:
        samp_rate = rate at which data is sampled per second 
        samp_num = total number of samples in a single data aquisition
        
    and returns:
       v_m = a list of all potentials sampled in the data acquisition
        
    '''
    
    
    # get a list of devices
    all_devices = list(nidaqmx.system.System.local().devices)
    
    # throw error if no devices are found or if multiple devices are found
    if (len(all_devices) == 0):
        raise IOError("No DAQ device found")
    if (not (len(all_devices) == 1)):
        warnings.warn("More than one DAQ device found. Using the first device. \
            Manually change dev_name to use other device")
        
    # otherwise use the first device that's found
    dev_name = all_devices[0].name
    
    # collect data, assign to v_m
    with Task() as task:
        
        # add input channel and set E range ( For CHEM174/274 potentiostat E range is always [-10,10] )
        task.ai_channels.add_ai_voltage_chan(dev_name + "/ai0", max_val=10, min_val=-10)
        
        '''
        # set the input voltage range to [-1,1] (not used in A2021)
        task.ai_channels.all.ai_min = -1
        task.ai_channels.all.ai_max = 1
        '''
        
        # set sampling rate and number of samples in acquisition 
        task.timing.cfg_samp_clk_timing(samp_rate, samps_per_chan=samp_num)
        
        # collect data from daq
        v_m = task.read(samp_num, timeout=nidaqmx.constants.WAIT_INFINITELY)
        
    return v_m
    

# Titration
Code block 1 will ask for the name of the experiment. This name will be used for the filenames of exported pltos or data. It will also create a list for storing the history of steps taken by the pump and potential measurements in each titration step

Code block 2 makes a potential acquisition and plots the data from all titration steps. After acquiring the data, the next aliquot of titrant is dispensed. You will run this code block repeatedly throughout the titration. 

You may change the parameters for each acquisition as well as the amount of titrant dispensed by the syringe pump.

**Do not rerun code block 3 before you want to end the titration**



In [ ]:
'''
rinse the syringe 
'''
# fill syringe
initialize_and_fill_and_expel_backlash()

# empty syringe
expel_and_close()

In [ ]:
"""
fill the syringe and expel backlash into wash vial 
"""

# fill syringe
initialize_and_fill_and_expel_backlash()

In [ ]:
"""
code block 1
"""
# ask for experiment name (will be used for exported plot and data)
measurement_name = input('name of experiment')
# stores cumulative 'steps' made by the pump motor when dispensing the titrant
history_steps = []
# stores samples from each potential acquisition
all_potential_measurements = []


In [ ]:
"""
code block 2
"""
# parameters for potential acquisition
samp_rate = 600
samp_num = 600

# Set the number of syringe steps of titrant dispensed in the next aliquot
num_steps = 100

# Check if desired amount to be dispensed exceeds syringe capacity
if len(history_steps) == 0:
    pass
# raise error if cumulative amount dispensed exceeds syringe capacity
elif (num_steps + history_steps[-1] > 11000):
        raise ValueError("Total Steps will exceed 11000. Decrease num_steps")
        
# If it's the first step, wait 5 seconds for user to immerse the tube into the liquid. 
if len(history_steps) == 0:
    history_steps.append(0)
    print('5 second delay for initial step.')
    time.sleep(5)

# take potential measurement and append data to storage list
curr_meas = get_potential_measurement(samp_rate, samp_num)
all_potential_measurements.append(curr_meas)

# calculate mean and standard deviation for all acquisitions
all_mean = [np.mean(meas_i) for meas_i in all_potential_measurements]
all_std = [np.std(meas_i) for meas_i in all_potential_measurements]

# plot results so far
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (14, 6))
# subplot 1 (sample vs potential of previous acquisiton)
ax1.scatter(np.arange(len(measured_potential)), measured_potential, facecolors='none', edgecolor = 'b')
ax1.set_title('Potential vs Sampling Number Previous Acquisition', fontsize = 16)
ax1.set_xlabel('Sampling Number', fontsize = 16)
ax1.set_ylabel('Potential / V', fontsize = 16)
# subplot 2 cumulative data collected at for all acquisitions
ax2.errorbar(history_steps, all_mean, yerr=all_std,  fmt='bo', fillstyle='none')
ax2.set_title('Titration', fontsize = 16)
ax2.set_xlabel('steps', fontsize = 16)
ax2.set_ylabel('Potential / V', fontsize = 16)

# print cumulative data collected
titration_data = pd.DataFrame({'step': history_steps , 'mean potential': all_mean, 'std dev': all_std})
print(titration_data)

# tell syringe pump to add titrant
add_titrant(num_steps)

# update cumulative total of steps added
history_steps.append(num_steps)
history_steps[-1] += history_steps[-2] 

# End titration and export plots and data

**Only run this when you want to end the titration**

When the titration is complete, code block 3 makes the final potential acquisition corresponding to the last
aliquot of titrant added. 

Code block 4 then replots the data and exports the plot and data acquired. 

In [1]:
"""
CODE BLOCK 3
"""

# parameters for potential measurements
samp_rate = 600
samp_num = 600

# take potential measurement and append data to array 
curr_meas = get_potential_measurement(samp_rate, samp_num)
all_potential_measurements.append(curr_meas)

# calculate mean and standard deviation of all acquisitions
all_mean = [np.mean(meas_i) for meas_i in all_potential_measurements]
all_std = [np.std(meas_i) for meas_i in all_potential_measurements]

NameError: name 'get_potential_measurement' is not defined

In [ ]:
"""
CODE BLOCK 4
"""

# plot results so far and save it. Make sure to change the image name for the second titration 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (14, 6))
# subplot 1 (sample vs potential of previous acquisiton)
ax1.scatter(np.arange(len(measured_potential)), measured_potential, facecolors='none', edgecolor = 'b')
ax1.set_title('Potential vs Sampling Number Previous Acquisition', fontsize = 16)
ax1.set_xlabel('Sampling Number', fontsize = 16)
ax1.set_ylabel('Potential (V)', fontsize = 16)
# subplot 2 cumulative data collected at for all acquisitions
ax2.errorbar(history_steps, all_mean, yerr=all_std,  fmt='bo', fillstyle='none')
ax2.set_title('Titration', fontsize = 16)
ax2.set_xlabel('steps', fontsize = 16)
ax2.set_ylabel('Potential (V)', fontsize = 16)

# Save figure in the current directory. Note this will overwrite existing images with the same filename 
plt.savefig(filename + '.png', dpi = 300, bbox_inches='tight')

# print cumulative data collected
titration_data = pd.DataFrame({'step': history_steps , 'mean potential': all_mean, 'std dev': all_std})
print(titration_data)

# Save titration data as txt. Make sure to change the name for each titration
titration_data.to_csv(filename + '.txt', index = False)


# Back titration 

After the titration is complete, code blocks 5-8 can be used to run a back titration. Code blocks 5-8 are very similar to code blocks 1-4 but for the back titration. The back titration data will be plotted on top of the initial titration data. 

In [ ]:
'''
rinse the syringe 
'''
# fill syringe
initialize_and_fill_and_expel_backlash()

# empty syringe
expel_and_close()

In [ ]:
"""
fill the syringe and expel backlash into wash vial 
"""

# fill syringe
initialize_and_fill_and_expel_backlash()

In [ ]:
'''
CODE BLOCK 5

initialize arrays for storing the history of steps added and potential measurements in back titration
'''

# stores cumulative 'steps' made by the pump motor when dispensing the titrant
rhistory_steps = []
# stores samples from each potential acquisition
rall_potential_measurements = []



In [ ]:
"""
CODE BLOCK 6

This block acquires data, then plots the data from all titration steps. 
After acquiring the data, the next aliquot of titrant is dispensed. 
This code block will be run repeatedly throughout the titration

Do not rerun code block 7 before the end of the back titration
"""

# parameters for potential acquisition
samp_rate = 600
samp_num = 600

# define step size or the amount of titrant dispensed in each step
num_steps = 100

# If it's the first step, wait 5 seconds for user to immerse the tube into the liquid. 
if len(rhistory_steps) == 0:
    rhistory_steps.append(history_steps[-1])
    print('5 second delay for initial step')
    time.sleep(5)

# take potential measurement and append data to storage arrays
rcurr_meas = get_potential_measurement(samp_rate, samp_num)
rall_potential_measurements.append(rcurr_meas)

# calculate mean and standard deviation of all acquisitions
rall_mean = [np.mean(meas_i) for meas_i in rall_potential_measurements]
rall_std = [np.std(meas_i) for meas_i in rall_potential_measurements]

# plot results so far 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (14, 6))
# subplot 1 (sample vs potential of previous acquisiton)
ax1.scatter(np.arange(len(measured_potential)), measured_potential, facecolors='none', edgecolor = 'b')
ax1.set_title('Potential vs Sampling Number Previous Acquisition', fontsize = 16)
ax1.set_xlabel('Sampling Number', fontsize = 16)
ax1.set_ylabel('Potential (V)', fontsize = 16)
# subplot 2 cumulative data collected at for all acquisitions
ax2.errorbar(rhistory_steps, rall_mean, yerr=rall_std,  fmt='ro', fillstyle='none', label = 'reverse')
ax2.errorbar(history_steps, all_mean, yerr=all_std,  fmt='bo',fillstyle='none', label = 'forward')
ax2.set_title('Titration', fontsize = 16)
ax2.set_xlabel('steps', fontsize = 16)
ax2.set_ylabel('Potential (V)', fontsize = 16)
ax2.legend()

# print cumulative data collected
titration_data = pd.DataFrame({'reverse step': rhistory_steps , 'mean potential': rall_mean, 'std dev': rall_std})
print(titration_data)

# tell syringe pump to add titrant
add_titrant(num_steps)

# get a cumulative total of reversed steps
rhistory_steps.append(-1*num_steps)
rhistory_steps[-1] += rhistory_steps[-2] 


# End back titration and export plots and data

**Only run this when you want to end the titration**

When the back titration is complete, code block 7 makes the final potential acquisition corresponding to the last
aliquot of titrant added. 

Code block 8 then replots the data and exports the plot and data acquired. 

In [ ]:
"""
CODE BLOCK 7
"""

# parameters for potential acquisition
samp_rate = 600
samp_num = 600

# take voltage measurement
rcurr_meas = get_potential_measurement(samp_rate, samp_num)
rall_potential_measurements.append(rcurr_meas)

# append data to arrays
rall_mean = [np.mean(meas_i) for meas_i in rall_potential_measurements]
rall_std = [np.std(meas_i) for meas_i in rall_potential_measurements]

In [ ]:
"""
CODE BLOCK 8
"""

# plot results so far and save it. Make sure to change the image name for the second titration 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (14, 6))
# subplot 1 (sample vs potential of previous acquisiton)
ax1.scatter(np.arange(len(measured_potential)), measured_potential)
ax1.set_title('Potential vs Sampling Number Previous Acquisition', fontsize = 16, facecolors='none', edgecolor = 'b')
ax1.set_xlabel('Sampling Number', fontsize = 16)
ax1.set_ylabel('Potential (V)', fontsize = 16)
# subplot 2 cumulative data collected at for all acquisitions
ax2.errorbar(rhistory_steps, rall_mean, yerr=rall_std,  fmt='ro', fillstyle='none', label = 'reverse')
ax2.errorbar(history_steps, all_mean, yerr=all_std,  fmt='bo',fillstyle='none', label = 'forward')
ax2.set_title('Vinegar Back Titration', fontsize = 16)
ax2.set_xlabel('steps', fontsize = 16)
ax2.set_ylabel('Potential (V)', fontsize = 16)
ax2.legend()

# Save figure in the current directory. Note this will overwrite existing images with the same filename 
plt.savefig('Vinegar back titration.png', dpi = 300, bbox_inches='tight')

# print cumulative data collected
titration_data = pd.DataFrame({'reverse step': rhistory_steps , 'mean potential': rall_mean, 'std dev': rall_std})
print(titration_data)

# Save titration data as txt. Make sure to change the name for each titration
titration_data.to_csv(filename + '.txt', index = False)

# Pump calibration and cleaning syringe pump

You will need to measure the amount of titrant added per step made by the syringe pump. It is recommended to do this gravimetriacally.

In [ ]:
"""
CODE BLOCK 16

Measure mass of titrant dispensed 
"""

# fill syringe
initialize_and_fill_and_expel_backlash()
# tell syringe pump to add titrant
add_titrant(6000)

In [ ]:
"""
CODE BLOCK 17

Clean syringe
"""

# fill syringe
initialize_and_fill_and_expel_backlash()
# empty syringe
expel_and_close()
